# 고금계 과제 1

20249433 최재필

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path

In [47]:
CWD = Path('.').resolve()
DATA_DIR = CWD / 'data'

In [48]:
from fndata import FnStockData
from fndata import FnMarketData

## Load data

In [49]:
fndata_path = DATA_DIR / '고금계과제1_v3.3_201301-202408.csv'
fnmkt_path = DATA_DIR / '고금계과제_시장수익률_201301-202408.csv'
rf_path = DATA_DIR / '통안채1년물_월평균_201301-202408.csv'

### 주식 데이터

In [50]:
# 데이터 모듈을 생성하며 기본 전처리들을 수행합니다.
fn = FnStockData(fndata_path)

In [51]:
# 사용 가능한 데이터를 확인합니다.
fn_items = fn.get_items()
fn_items

array(['종가(원)', '수정계수', '수정주가(원)', '수익률 (1개월)(%)', 'FnGuide Sector',
       '거래정지여부', '관리종목여부', '보통주자본금(천원)', '자본잉여금(천원)', '이익잉여금(천원)',
       '자기주식(천원)', '이연법인세부채(천원)', '매출액(천원)', '매출원가(천원)', '이자비용(천원)',
       '영업이익(천원)', '총자산(천원)', '기말발행주식수 (보통)(주)'], dtype=object)

In [52]:
# 분석 기간의 데이터 유니버스를 확인합니다. (금융업종, 거래정지, 관리종목 제외)
univ_list = fn.get_universe()
univ_list

array(['A000020', 'A000040', 'A000050', ..., 'A001260', 'A019660',
       'A023460'], dtype=object)

In [53]:
# long format (전부)

df = fn.get_data()
df.head()

Item Name          FnGuide Sector 거래정지여부 관리종목여부  기말발행주식수 (보통)(주)  \
date       Symbol                                                  
2013-01-31 A000020             의료     정상     정상       27931470.0   
           A000040          경기소비재     정상     정상      119341379.0   
           A000050          경기소비재     정상     정상        2492298.0   
           A000070            산업재     정상     정상        7704997.0   
           A000080          필수소비재     정상     정상       69477714.0   

Item Name                매출액(천원)      매출원가(천원)   보통주자본금(천원)  수익률 (1개월)(%)  \
date       Symbol                                                           
2013-01-31 A000020  2.202406e+08  1.165344e+08   27931470.0         -0.17   
           A000040  9.955327e+07  9.199679e+07   59670690.0          0.20   
           A000050  3.471896e+08  2.514682e+08   12461490.0         -3.46   
           A000070  2.329145e+09  2.025952e+09   40852325.0         -4.51   
           A000080  1.897469e+09  1.074666e+09  363168055.0          5.26   

Item Name           수정계수  수정주가(원)     영업이익(천원)  이연법인세부채(천원)    이익잉여금(천원)  \
date       Symbol                                                          
2013-01-31 A000020   1.0   6030.0    2062220.0          0.0  170147330.0   
           A000040   1.0   8727.0   -3486600.0    7181340.0    1298918.0   
           A000050   1.0   7874.0   36031739.0   90652345.0  612310562.0   
           A000070   1.0  67800.0    2472122.0  110818108.0  871019983.0   
           A000080   1.0  32000.0  161051461.0   23405084.0  422540903.0   

Item Name             이자비용(천원)     자기주식(천원)    자본잉여금(천원)    종가(원)  \
date       Symbol                                                   
2013-01-31 A000020    384335.0          0.0   26919758.0   6030.0   
           A000040    813333.0      -5044.0    3884892.0    495.0   
           A000050  13944625.0          0.0   13991400.0  86500.0   
           A000070  17176000.0   -9982000.0  355855971.0  67800.0   
           A000080  56438096.0 -122693570.0  709299321.0  32000.0   

Item Name                총자산(천원)  
date       Symbol                 
2013-01-31 A000020  3.161655e+08  
           A000040  1.169316e+08  
           A000050  1.296989e+09  
           A000070  2.595068e+09  
           A000080  3.553147e+09

In [64]:
# wide format

monthly_returns = fn.get_data('수익률 (1개월)(%)')
monthly_returns.head()

Symbol,A000020,A000040,A000050,A000070,A000080,A000100,A000120,A000140,A000150,A000180,...,A014940,A053810,A007390,A002410,A037950,A038880,A015020,A001260,A019660,A023460
date,,,,,,,,,,,,,,,,,,,,,
2013-01-31,-0.0017,0.0020,-0.0346,-0.0451,0.0526,0.0607,0.1940,-0.0122,-0.0039,-0.0559,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-02-28,0.0033,-0.0470,0.0537,0.0251,0.0791,-0.0028,-0.0295,-0.0414,0.0078,0.0520,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-03-31,0.0133,-0.0064,0.0288,0.0187,-0.0073,0.0700,-0.1452,0.0432,0.0039,0.1621,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-30,0.1047,0.0000,0.2594,0.3239,-0.0473,0.0995,0.0783,-0.1302,-0.0502,0.0946,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-05-31,0.0267,0.0475,0.1667,-0.0598,-0.0140,-0.1167,-0.0283,-0.0238,0.1545,0.0367,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 시장 데이터

In [54]:
# 데이터 모듈을 생성하며 기본 전처리들을 수행합니다.
fnmkt = FnMarketData(fnmkt_path)

In [55]:
mkt = fnmkt.get_data(format='wide')

mkf2000 = mkt['MKF2000']
krx300 = mkt['KRX 300']

### 무위험 이자율

In [56]:
rf = pd.read_csv(rf_path)
rf.columns = ['date', 'rf']
rf.set_index('date', inplace=True)
rf.index = pd.to_datetime(rf.index, format='%Y/%m') + pd.offsets.MonthEnd(0)

In [58]:
rf_m = (1 + rf / 100) ** (1 / 12) - 1 # 월율화
rf_m.head()

,rf
date,
2013-01-31,0.002239
2013-02-28,0.002214
2013-03-31,0.002149
2013-04-30,0.002125
2013-05-31,0.002125


## 팩터 데이터 생성

2d wide format을 이용

### 1. Mkt Cap

= 기업규모 = 시장가치 = 시가총액 = 보통주 주가 * 발행주식수

In [60]:
raw_close = fn.get_data('종가(원)')
common_shares_outstanding = fn.get_data('기말발행주식수 (보통)(주)')

In [61]:
FF_size = raw_close * common_shares_outstanding

### 2. B/M

자기자본 장부가치 / 시장가치

#### 2.1 Book 계산

= 자기자본 장부가치 = t-1년 12월말 보통주 자본금 + 자본잉여금 + 이익잉여금 + 자기주식 + 이연법인세 부채

** 장부가치 음수인 자본잠식 상태의 기업들 분석에서 제외

In [73]:
common_stock = fn.get_data('보통주자본금(천원)') # 천원단위 아님
capital_surplus = fn.get_data('자본잉여금(천원)')
retained_earnings = fn.get_data('이익잉여금(천원)')
treasury_stock = fn.get_data('자기주식(천원)')
deferred_tax_liabilities = fn.get_data('이연법인세부채(천원)')

In [70]:
print(f'''
common_stock: {common_stock.shape}
capital_surplus: {capital_surplus.shape}
retained_earnings: {retained_earnings.shape}
treasury_stock: {treasury_stock.shape}
deferred_tax_liabilities: {deferred_tax_liabilities.shape}
''')


common_stock: (138, 2616)
capital_surplus: (138, 2616)
retained_earnings: (138, 2616)
treasury_stock: (138, 2616)
deferred_tax_liabilities: (138, 2616)



******** 발견한 것: 

재무데이터와 `monthly_return` shape이 다르다. 

재무데이터는 6월30일까지만 불러와져 있다. 그것이 재무제표 업데이트하는 시점인 듯. 

******** 발견한 것: 

재무데이터, 이미 t-1시점 값으로 밀어져있다. 

그냥 쓰는 것이 맞는 것으로 보인다. 

In [71]:
# # t-1년도의 값으로 ffill 

# eoy = (common_stock.index.month == 12)

# common_stock = common_stock.loc[eoy, :].reindex(
#     index=common_stock.index, method='ffill'
# )

# capital_surplus = capital_surplus.loc[eoy, :].reindex(
#     index=capital_surplus.index, method='ffill'
# )

# retained_earnings = retained_earnings.loc[eoy, :].reindex(
#     index=retained_earnings.index, method='ffill'
# )

# treasury_stock = treasury_stock.loc[eoy, :].reindex(
#     index=treasury_stock.index, method='ffill'
# )

# deferred_tax_liabilities = deferred_tax_liabilities.loc[eoy, :].reindex(
#     index=deferred_tax_liabilities.index, method='ffill'
# )

********* nan인 value들 있음. 

특히 dtl 같은 것은 많은 기업이 0인 경우가 있기 때문에 `fillna(0)` 하는 것이 맞아보임. 

treasury stock도 마찬가지임

capital surplus도. 

`ffill` 하는 것은 틀린 선택 같음. 

In [96]:
common_stock = common_stock.fillna(0)
capital_surplus = capital_surplus.fillna(0)
retained_earnings = retained_earnings.fillna(0)
treasury_stock = treasury_stock.fillna(0)
deferred_tax_liabilities = deferred_tax_liabilities.fillna(0)

In [97]:
# fillna(0) 없이 처리하면 하나라도 nan이면 모두 nan으로 처리됨

# book_equity = common_stock + capital_surplus + retained_earnings - treasury_stock - deferred_tax_liabilities
book_equity = common_stock + capital_surplus + retained_earnings + treasury_stock + deferred_tax_liabilities

*********** 이거 공식 맞나?? 

book equity = common stock + capital surplus + retained earnings - treasury stock 

이고, 

dtl 은 retained earning에 영향을 직접 준다고 함. 

#### 2.2 B/M 계산

In [102]:
mkt_cap_12 = FF_size.loc[FF_size.index.month == 12, :].reindex(
    index=common_stock.index, method='ffill'
) # market cap은 재무데이터가 아니라 가장 최근 9월까지도 다 채워져있음.
# 재무데이터 길이와 맞추도록 reindex

In [104]:
FF_bm = book_equity / mkt_cap_12

### 3. 수익성

= ROE = Net Income / Equity 

### 3.1 Operating Profit

= t-1년 12월 말의 매출액 - 매출원가 - 이자비용 - 판매및관리비 차감한 

영업이익 

In [106]:
# sales_revenue = fn.get_data('매출액(천원)')
# cost_of_goods_sold = fn.get_data('매출원가(천원)')
# interest_expense = fn.get_data('이자비용(천원)')
operating_profit = fn.get_data('영업이익(천원)')

********** 매출액조차 분기별로 변하지 않고 연간으로 변한다.

이것도 이미 작년 12월 값으로 ffill 된 것 같음. 별도 처리 필요 x 

#### 3.2 ROE 계산

NI, Equity 둘 다 t-1년 12월 말 기준

In [112]:
FF_roe = operating_profit / book_equity

### 4. 자본투자

= (t-1년 12월 총자산 - t-2년 12월 총자산) / t-2년 12월 총자산 = 즉, 총자산의 증가율

In [111]:
total_asset = fn.get_data('총자산(천원)')

In [113]:
FF_inv = (total_asset - total_asset.shift(12)) / total_asset.shift(12)

### 5. UMD

= Up Minus Down = (전월말 주가 - 1년전 월말 주가 ) / 1년전 월말 주가

** 모멘텀의 경우 1개월마다 리밸런싱, 상위 30%가 winner / 하위 30%가 loser

### 6. STR

= reversal 

최근 월의 수익률을 기반으로 구축 ? 

## 팩터 포트폴리오 생성

### 팩터 수익률 계산

### Size-B/M 포트폴리오

- Size, B/M으로 double sort

## Devil's in HML's detail 

- Asness의 주장대로 B/M 계산시 B는 작년 12월을 쓰더라도 M은 최근 6월 주가를 사용
- 변경된 B/M 포트폴리오를 통해 결과 재산출